In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import  datasets, transforms

In [2]:
BATCH_SIZE = 512
EPOCHS = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
pipeline=transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.1307,),(0.3081,))]
)

In [4]:
from torch.utils.data import DataLoader

train_set=datasets.MNIST(
    root="../data",
    train=True,
    download=True,
    transform=pipeline
)

test_set=datasets.MNIST(
    root="../data",
    train=False,
    download=True,
    transform=pipeline
)

train_loader=DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader=DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)


In [5]:
def train(model,device,train_loader,optimizer,epoch):
    model.train()
    for batch_idx,(data,target) in enumerate(train_loader):
        data,target=data.to(device),target.to(device)
        optimizer.zero_grad()
        output=model(data)
        loss=F.cross_entropy(output,target)
        loss.backward()
        optimizer.step()
    # 把输出内容保存在 “CNN.txt” 文件中
    with open("../result/LR.txt", "a") as f:
        f.write(f"Epoch {epoch}, Loss: {loss.item()}\n")
        

In [6]:
def test(model,device,test_loader):
    model.eval()
    test_loss=0
    correct=0
    with torch.no_grad():
        for data,target in test_loader:
            data,target=data.to(device),target.to(device)
            output=model(data)
            test_loss+=F.cross_entropy(output,target,reduction="sum").item()
            pred=output.max(1,keepdim=True)[1]
            correct+=pred.eq(target.view_as(pred)).sum().item()
    test_loss/=len(test_loader.dataset)
    with open("../result/LR.txt","a") as f:
        f.write("\nTest set: Average loss: %f, Accuracy: %d/%d (%f)\n" % (
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))
    

In [7]:
import time
def train_test(model,device,train_loader,test_loader,optimizer,epoch):
    for epoch in range(1,epoch+1):
        start=time.time()
        train(model,device,train_loader,optimizer,epoch)
        end=time.time()
        with open("../result/LR.txt","a") as f:
            f.write("Train Time: %f\n" % (end-start))
        start=time.time()
        test(model,device,test_loader)
        end=time.time()
        with open("../result/LR.txt","a") as f:
            f.write("Test Time: %f\n" % (end-start))
            

In [8]:
class LR(nn.Module):
    def __init__(self):
        super(LR,self).__init__()
        self.fc=nn.Linear(28*28,10)
        
    def forward(self,x):
        x=x.view(-1,784)
        x=self.fc(x)
        return F.log_softmax(x,dim=1)

In [9]:
def run():
    model=LR().to(DEVICE)
    print(model)
    optimizer=optim.Adam(model.parameters())
    train_test(model,DEVICE,train_loader,test_loader,optimizer,EPOCHS)
    

In [10]:
if __name__ == "__main__":
    run()

LR(
  (fc): Linear(in_features=784, out_features=10, bias=True)
)
